In [ ]:
import cmipld
from cmipld.utils.offline import LD_server, replace_loader
from pyld import jsonld

In [ ]:
import cmipld
import importlib


repo_url = cmipld.utils.git.url()
io_url = cmipld.utils.git.url2io(repo_url)

whoami = cmipld.reverse_mapping()[io_url]
print('-'*50)
print(f'Parsing repo: {whoami}')
print(f'Location: {repo_url}')
print(f'Github IO link: {io_url}')
print('-'*50)

'''
To keep this set of functions generic, we create separate files containing all relevant functions for a repository to be processed. 
When running this script we determine the parent repository, and consequently its prefix. This is then used to grab all relevant functions for processing. 

Example usage: getattr(this,'activity')({'activity':[{"label":1,"id":'one'}]})

'''
# Dynamically import the target module and assign it to a variable
module_path = f"cmipld.cvs.{whoami}" # go to CMIPLD/CVs/<value of whoami>.py for functions
this = importlib.import_module(module_path)



In [ ]:
# Set up offline override for the current repository

ldpath = cmipld.utils.git.ldpath()


# create a temporary directory containing (a copy of) the current repository
# localserver = LD_server(copy = [[ldpath,'current_repo']])
repos= { 
        'https://wcrp-cmip.github.io/WCRP-universe/': 'universal',
        'https://wcrp-cmip.github.io/MIP-variables/': 'variables',
        'https://wcrp-cmip.github.io/CMIP6Plus_CVs/': 'cmip6plus'
        }

localserver = LD_server(repos = repos.items(),copy = [[ldpath,whoami]],override='y')

# localserver = LD_server(copy = [[ldpath,'cmip6plus']])


localhost = localserver.start_server(8080)


In [ ]:
# 
replace_loader(localhost,[list(i) for i in repos.items()])


In [5]:
from collections import OrderedDict

CV = OrderedDict()




In [ ]:

for item in [
    'repo',
    'mip-era',
    'license',
    'activity',
    'drs',
    'product',
    'required-global-attributes',
    # 'tables',
    ]:
    
    print(item)
    # CV[item] = getattr(locals(),item)
    ld  = cmipld.processor.expand_document(f'{io_url}project/{item}.json', depth=3)[0]
    CV[item] = getattr(this,item)(ld)


In [6]:
# adding the index properties
CV.update(cmipld.processor.get(f'{io_url}/project/index.json')[0]['index'])

In [ ]:
for universe_item in ['realm','frequency']:
    # 'grid-label''nominal'
    print(universe_item)
    path = f'{cmipld.mapping["universal"]}{universe_item}/graph.jsonld'
    ld  = cmipld.jsonld.compact(path,path)['@graph']
    CV[universe_item] = getattr(this,universe_item)(ld)

In [8]:
path = f'{cmipld.mapping["universal"]}resolution/graph.jsonld'
data  = cmipld.jsonld.compact(path,path)['@graph']

CV['nominal_resolution'] = [f"{r['value']} {r['unit']}" for r in sorted(data, key=lambda x: float(x['value']) if x['value'].replace('.', '', 1).isdigit() else float('inf'))]



In [ ]:
exp = cmipld.processor.get(f'{whoami}:experiment/graph.jsonld')
exp

In [ ]:
CV

In [ ]:
e1 = exp[1]
# del e1['id'], e1['type']
for k,v in e1.items():
    if isinstance(v,dict) and 'type' in v:
        print(k)
        e1[k] = v['label']
e1


In [ ]:
import cmipld
from pyld import jsonld
from pprint import pprint
pprint(jsonld.expand('https://wcrp-cmip.github.io/CMIP6Plus_CVs/experiment/1pctco2.json'))

In [ ]:
import cmipld
from pprint import pprint
pprint(cmipld.processor.get(f'cmip6plus:experiment/1pctco2.json',depth=2))

In [ ]:
pprint(jsonld.frame('https://wcrp-cmip.github.io/CMIP6Plus_CVs/experiment/1pctco2.json',{"@context":'https://wcrp-cmip.github.io/CMIP6Plus_CVs/experiment/1pctco2.json',"@embed":"@always"}))

In [ ]:
pprint(jsonld.expand(f'{io_url}project/activity.json'))

In [ ]:
import glob
import json
ctx = json.load(open('_context_'))['@context'][1]
for i,j in ctx.items(): 
    if '@' not in i:
        data = json.load(open(f"{i.replace('_','-')}.json"))
        data['@context']    =   ['_context_',{i:j}]
        json.dump(data, open(f"{i.replace('_','-')}.json",'w'),indent=4)

In [ ]:

pprint(CV)

In [2]:

from pyld import jsonld
from pprint import pprint
import json

def jprint(c):
    print(json.dumps(c,indent=4))


group = [
    'https://wcrp-cmip.github.io/CMIP6Plus_CVs/experiment/graph.jsonld',
    'https://wcrp-cmip.github.io/CMIP6Plus_CVs/sub-experiment/graph.jsonld',
    'https://wcrp-cmip.github.io/WCRP-universe/source-type/graph.jsonld',
    'https://wcrp-cmip.github.io/WCRP-universe/activity/graph.jsonld',
]



# content = {'@graph':[jsonld.expand(i) for i in group]}

# content = jsonld.expand(group)

# content.keys()

In [ ]:
import cmipld
cmipld.processor.extract_dependencies(group[0],graph=True)


In [ ]:
out = jsonld.frame(content,{"@context":'https://wcrp-cmip.github.io/CMIP6Plus_CVs/experiment/_context_',"type":["cmip6plus", "wcrp-experiment"],"@required":"True" })

nout = [i for i in out['@graph'] if 'wcrp:experiment' in i['type']]

jprint(nout)

In [ ]:
jprint(nout[7]['model_realms'])
nout[7]